In [1]:
import torch
import torchvision.transforms as transforms
import torchvision
import sklearn
from sklearn import tree
from matplotlib import pyplot as plt

In [2]:
BS = 50000
trainset = torchvision.datasets.MNIST(root='~/Private/data', train=True,
                                        download=False, transform=transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BS,
                                        shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='~/Private/data', train=False,
                                    download=False, transform=transforms.ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                            shuffle=False, num_workers=2)

In [23]:
clf = tree.DecisionTreeClassifier(max_depth=4)
for images, labels in trainloader:
    clf.fit(images.view(images.size(0),-1).numpy(), labels.numpy())

In [21]:
total = 0
correct = 0
for images, labels in testloader:
    images, labels = images.view(images.size(0),-1).numpy(), labels.numpy()
    predictions = clf.predict(images)
    correction = predictions == labels
    correct += correction.sum()
    total += len(correction)
print(correct/total)

0.642


In [5]:
f = open("lol.txt","w+")
a = tree.export_text(clf)
f.write(a)
f.close()

In [6]:
lines = a.split("\n")
row = [0 for _ in range(784)]
j = 0
for l in lines:
    app = 10 - (l.find("feature") - 1)//4
    if app == 11:
        continue
    left = l.find("_") + 1
    right = l.find("<=") if l.find("<=") != -1 else l.find(">")
    index = int(l[left:right])
    row[index] += pow(2, app)
    

In [7]:
total = 0
col = 0
for i in row:
    if i > 0:
        total += i
        col += 1

In [8]:
print(total/(col*1024))

0.031856385379061374


In [9]:
print(col)

277


In [74]:
import graphviz 
# dot_data = dot_data = tree.export_graphviz(clf, out_file="dot", 
#                     filled=True, rounded=True,  
#                     special_characters=True) 
f = open("dot","r")
new_d = f.read()
new_d = new_d.splitlines()
for i in range(3,len(new_d)):
    l = new_d[i]
    left = l.find("<br/>")
    right = l.find("]")
    if left != -1 and right != -1 and l.find("X")!=-1:
        new_d[i] = (l[:left] + l[right:]).replace("]", "")[:-2] + "];\n"
new_d = "".join(new_d)
graph = graphviz.Source(new_d) 
graph.render("iris") 

'iris.pdf'

In [27]:
import dtreeviz
viz = dtreeviz.trees.dtreeviz(clf, images.view(-1,784).numpy(), labels.numpy(), list(range(784)), list(range(10)))

In [28]:
viz.view()